In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import BertTokenizer
from transformers import RobertaTokenizer
# from transformers import GPT2TokenizerFast
from transformers import AlbertTokenizer
from transformers import DistilBertTokenizer
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification
from transformers import RobertaForSequenceClassification
# from transformers import GPT2ForSequenceClassification
from transformers import AlbertForSequenceClassification
from transformers import DistilBertForSequenceClassification
import warnings

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ht_small=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/small.csv')
ht_small=ht_small.dropna()
ht_big=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/big.csv')
ht_big=ht_big.dropna()
marianmt_m2m100=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/MarianMT_DE/MarianMT_DE_M2M100_KO.csv')
marianmt_m2m100=marianmt_m2m100.dropna()
marianmt_mbart=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/MarianMT_DE/MarianMT_DE_MBart_KO.csv')
marianmt_mbart=marianmt_mbart.dropna()
m2m100_mbart=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/M2M100_DE/M2M100_DE_MBart_KO.csv')
m2m100_mbart=m2m100_mbart.dropna()
m2m100_m2m100=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/M2M100_DE/M2M100_DE_M2M100_KO.csv')
m2m100_m2m100=m2m100_m2m100.dropna()
mbart_m2m100=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/MBart_DE/MBart_DE_M2M100_KO.csv')
mbart_m2m100=mbart_m2m100.dropna()
mbart_mbart=pd.read_csv('/Volumes/T7/mt-hemingway/data/hypothesis/exp2/MBart_DE/MBart_DE_MBart_KO.csv')
mbart_mbart=mbart_mbart.dropna()

In [3]:
small_marianmt_m2m100=[ht_small.sample(ht_small.shape[0]), marianmt_m2m100.sample(marianmt_m2m100.shape[0])]
small_marianmt_m2m100=pd.concat(small_marianmt_m2m100)
small_marianmt_mbart=[ht_small.sample(ht_small.shape[0]), marianmt_mbart.sample(marianmt_mbart.shape[0])]
small_marianmt_mbart=pd.concat(small_marianmt_mbart)
small_m2m100_mbart=[ht_small.sample(ht_small.shape[0]), m2m100_mbart.sample(m2m100_mbart.shape[0])]
small_m2m100_mbart=pd.concat(small_m2m100_mbart)
small_m2m100_m2m100=[ht_small.sample(ht_small.shape[0]), m2m100_m2m100.sample(m2m100_m2m100.shape[0])]
small_m2m100_m2m100=pd.concat(small_m2m100_m2m100)
small_mbart_m2m100=[ht_small.sample(ht_small.shape[0]), mbart_m2m100.sample(mbart_m2m100.shape[0])]
small_mbart_m2m100=pd.concat(small_mbart_m2m100)
small_mbart_mbart=[ht_small.sample(ht_small.shape[0]), mbart_mbart.sample(mbart_mbart.shape[0])]
small_mbart_mbart=pd.concat(small_mbart_mbart)
big_marianmt_m2m100=[ht_big.sample(ht_big.shape[0]), marianmt_m2m100.sample(marianmt_m2m100.shape[0])]
big_marianmt_m2m100=pd.concat(big_marianmt_m2m100)
big_marianmt_mbart=[ht_big.sample(ht_big.shape[0]), marianmt_mbart.sample(marianmt_mbart.shape[0])]
big_marianmt_mbart=pd.concat(big_marianmt_mbart)
big_m2m100_mbart=[ht_big.sample(ht_big.shape[0]), m2m100_mbart.sample(m2m100_mbart.shape[0])]
big_m2m100_mbart=pd.concat(big_m2m100_mbart)
big_m2m100_m2m100=[ht_big.sample(ht_big.shape[0]), m2m100_m2m100.sample(m2m100_m2m100.shape[0])]
big_m2m100_m2m100=pd.concat(big_m2m100_m2m100)
big_mbart_m2m100=[ht_big.sample(ht_big.shape[0]), mbart_m2m100.sample(mbart_m2m100.shape[0])]
big_mbart_m2m100=pd.concat(big_mbart_m2m100)
big_mbart_mbart=[ht_big.sample(ht_big.shape[0]), mbart_mbart.sample(mbart_mbart.shape[0])]
big_mbart_mbart=pd.concat(big_mbart_mbart)

In [4]:
def convert_label(data:pd.DataFrame):
    data['label']=data['label'].replace({data['label'].groupby(data['label']).count().keys()[0]:0, data['label'].groupby(data['label']).count().keys()[1]:1})
    return data

In [5]:
small_marianmt_m2m100=convert_label(small_marianmt_m2m100)
small_marianmt_mbart=convert_label(small_marianmt_mbart)
small_m2m100_mbart=convert_label(small_m2m100_mbart)
small_m2m100_m2m100=convert_label(small_m2m100_m2m100)
small_mbart_m2m100=convert_label(small_mbart_m2m100)
small_mbart_mbart=convert_label(small_mbart_mbart)
big_marianmt_m2m100=convert_label(big_marianmt_m2m100)
big_marianmt_mbart=convert_label(big_marianmt_mbart)
big_m2m100_mbart=convert_label(big_m2m100_mbart)
big_m2m100_m2m100=convert_label(big_m2m100_m2m100)
big_mbart_m2m100=convert_label(big_mbart_m2m100)
big_mbart_mbart=convert_label(big_mbart_mbart)

In [6]:
%%time
X=list(big_mbart_mbart['text'])
y=list(big_mbart_mbart['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, stratify=y, random_state=42)

CPU times: user 1.56 ms, sys: 1.06 ms, total: 2.62 ms
Wall time: 1.83 ms


In [7]:
bert_tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
albert_tokenizer=AlbertTokenizer.from_pretrained('albert-base-v2')
roberta_tokenizer=RobertaTokenizer.from_pretrained('roberta-base')
bert_model=BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
albert_model=AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)
roberta_model=RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
import warnings

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [8]:
device=torch.device('mps')
bert_model.to(device)
roberta_model.to(torch.device('cpu'))
albert_model.to(torch.device(device))
device

device(type='mps')

In [9]:
X_train_tokenized_bert=bert_tokenizer(X_train, padding=True, truncation=True)#, max_length=512)
X_test_tokenized_bert=bert_tokenizer(X_test, padding=True, truncation=True)#, max_lenght=512)
X_train_tokenized_roberta=roberta_tokenizer(X_train, padding=True, truncation=True)#, max_length=512)
X_test_tokenized_roberta=roberta_tokenizer(X_test, padding=True, truncation=True)#, max_lenght=512)
X_train_tokenized_albert=albert_tokenizer(X_train, padding=True, truncation=True)#, max_length=512)
X_test_tokenized_albert=albert_tokenizer(X_test, padding=True, truncation=True)#, max_lenght=512)

In [10]:
#create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [11]:
train_dataset_bert=Dataset(X_train_tokenized_bert, y_train)
test_dataset_bert=Dataset(X_test_tokenized_bert, y_test)
train_dataset_roberta=Dataset(X_train_tokenized_roberta, y_train)
test_dataset_roberta=Dataset(X_test_tokenized_roberta, y_test)
train_dataset_albert=Dataset(X_train_tokenized_albert, y_train)
test_dataset_albert=Dataset(X_test_tokenized_albert, y_test)

In [12]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred=np.argmax(pred, axis=1)
    accuracy=accuracy_score(y_true=labels, y_pred=pred)
    recall=recall_score(y_true=labels, y_pred=pred)
    precision=precision_score(y_true=labels, y_pred=pred)
    f1=f1_score(y_true=labels, y_pred=pred)
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

In [13]:
#define Trainer
args_bert=TrainingArguments(
    output_dir='output',
    num_train_epochs=1,#higher may get the higer accuracy
    per_device_train_batch_size=8,
    use_mps_device=True
)
trainer_bert=Trainer(
    model=bert_model,
    args=args_bert,
    train_dataset=train_dataset_bert,
    eval_dataset=test_dataset_bert,
    compute_metrics=compute_metrics,
)

In [14]:
trainer_bert.train()

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1220
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 153
  Number of trainable parameters = 109483778
100%|██████████| 153/153 [01:29<00:00,  1.82it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 153/153 [01:29<00:00,  1.71it/s]

{'train_runtime': 89.4315, 'train_samples_per_second': 13.642, 'train_steps_per_second': 1.711, 'train_loss': 0.4155426025390625, 'epoch': 1.0}


TrainOutput(global_step=153, training_loss=0.4155426025390625, metrics={'train_runtime': 89.4315, 'train_samples_per_second': 13.642, 'train_steps_per_second': 1.711, 'train_loss': 0.4155426025390625, 'epoch': 1.0})

In [15]:
trainer_bert.evaluate()

***** Running Evaluation *****
  Num examples = 306
  Batch size = 8
100%|██████████| 39/39 [00:06<00:00,  6.46it/s]

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.25748103857040405,
 'eval_accuracy': 0.9117647058823529,
 'eval_precision': 0.8757396449704142,
 'eval_recall': 0.961038961038961,
 'eval_f1': 0.9164086687306501,
 'eval_runtime': 6.6715,
 'eval_samples_per_second': 45.866,
 'eval_steps_per_second': 5.846,
 'epoch': 1.0}

In [16]:
args_roberta=TrainingArguments(
    output_dir='output',
    num_train_epochs=1,#higher may get the higer accuracy
    per_device_train_batch_size=8,
    # use_mps_device=True,
    # pytorch_device='cpu',
)
trainer_roberta=Trainer(
    model=roberta_model,
    args=args_roberta,
    train_dataset=train_dataset_roberta,
    eval_dataset=test_dataset_roberta,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [17]:
trainer_roberta.train()

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1220
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 153
  Number of trainable parameters = 124647170
100%|██████████| 153/153 [13:32<00:00,  4.60s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 153/153 [13:32<00:00,  5.31s/it]

{'train_runtime': 812.3285, 'train_samples_per_second': 1.502, 'train_steps_per_second': 0.188, 'train_loss': 0.38239844328437755, 'epoch': 1.0}


TrainOutput(global_step=153, training_loss=0.38239844328437755, metrics={'train_runtime': 812.3285, 'train_samples_per_second': 1.502, 'train_steps_per_second': 0.188, 'train_loss': 0.38239844328437755, 'epoch': 1.0})

In [18]:
trainer_roberta.evaluate()

***** Running Evaluation *****
  Num examples = 306
  Batch size = 8
100%|██████████| 39/39 [00:38<00:00,  1.01it/s]

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.10036637634038925,
 'eval_accuracy': 0.9705882352941176,
 'eval_precision': 0.9617834394904459,
 'eval_recall': 0.9805194805194806,
 'eval_f1': 0.9710610932475885,
 'eval_runtime': 39.8919,
 'eval_samples_per_second': 7.671,
 'eval_steps_per_second': 0.978,
 'epoch': 1.0}

In [13]:
args_albert=TrainingArguments(
    output_dir='output',
    num_train_epochs=1,#higher may get the higer accuracy
    per_device_train_batch_size=8,
    use_mps_device=True
)
trainer_albert=Trainer(
    model=albert_model,
    args=args_albert,
    train_dataset=train_dataset_albert,
    eval_dataset=test_dataset_albert,
    compute_metrics=compute_metrics,
)

In [14]:
trainer_albert.train()

/Users/mstudio/miniconda3/envs/py3.10/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1220
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 153
  Number of trainable parameters = 11685122
100%|██████████| 153/153 [01:17<00:00,  2.07it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 153/153 [01:17<00:00,  1.98it/s]

{'train_runtime': 77.3033, 'train_samples_per_second': 15.782, 'train_steps_per_second': 1.979, 'train_loss': 0.4446257048962163, 'epoch': 1.0}


TrainOutput(global_step=153, training_loss=0.4446257048962163, metrics={'train_runtime': 77.3033, 'train_samples_per_second': 15.782, 'train_steps_per_second': 1.979, 'train_loss': 0.4446257048962163, 'epoch': 1.0})

In [15]:
trainer_albert.evaluate()

***** Running Evaluation *****
  Num examples = 306
  Batch size = 8
100%|██████████| 39/39 [00:06<00:00,  5.71it/s]

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.36952126026153564,
 'eval_accuracy': 0.8529411764705882,
 'eval_precision': 0.8224852071005917,
 'eval_recall': 0.9025974025974026,
 'eval_f1': 0.8606811145510836,
 'eval_runtime': 7.4458,
 'eval_samples_per_second': 41.097,
 'eval_steps_per_second': 5.238,
 'epoch': 1.0}